# old puzzel data last year

In [1]:
import puzzelsData as BPD

contains converToLonSrt



In [2]:
res = [ el for el in dir(BPD) if not el.startswith('__')]
res


['L17',
 'P6x6Mor_1',
 'P6x6ma_1',
 'Tubantia8aug',
 'convertToLStr',
 'm1_8x8',
 'needezeniet',
 'normal8',
 'oplossingvanneedezeniet',
 'p4x4moeilijk',
 'tub190918',
 'tub6sol',
 'tubantia11apr18',
 'tubantia15aug',
 'tubantia18juli',
 'tubantia20juni',
 'tubantia22aug',
 'tubantia22augNG',
 'tubantia25juli',
 'tubantia27_6',
 'tubantia28aug',
 'tubantia2aug',
 'tubantia_11juli',
 'zes_1',
 'zes_1_test']

In [3]:
TestBP = BPD.convertToLStr(BPD.tubantia20juni)
TestBP

'..10......00...0........0.11..1...........0...01.....0...1.10....0.......1...........0..1..0.....1..'

In [4]:
BPD.tubantia15aug #from last year ...

['1..1..11..',
 '..........',
 '0.11.11.0.',
 '0.1...0..1',
 '...0......',
 '1.1.......',
 '.0........',
 '....11..0.',
 '11..11...0',
 '.......1..']

# Tubantia BP 
try to solve with Z3

https://pclubiiti.github.io/blog/Inroduction-toSAT-solving/

```
Each binary puzzle should be solved according to the following rules:

Each box should contain a zero or a one.
No more than two similar numbers next to or below each other are allowed.
Each row and each column should contain an equal number of zeros and ones.
Each row is unique and each column is unique. 

PKHG>not all puzzels of my local newspaper obeyed the next info:

Each binary puzzle does only have one solution. 

PKHG: is this always true if there is precisely ONE solution??
You can always find this solution without guessing. 

```
From here the commands in BPsolverZ3 ... are used ;-)

In [5]:
from z3 import *

```
rr = BPD.tubantia15aug
type(rr) == type([1])
```
problem in the def?

In [6]:
def BPsolverZ3(puzzel, tussenresultaat = True):
    """ binary puzzel solver met Z3 """
    print("taken from \n\
          https://pclubiiti.github.io/blog/Inroduction-toSAT-solving/")
    size = 10 
    """ gives error???
    if type(puzzel) == type([1]):
        try:
            puzzel = convertToLStr(puzzel)
            from math import sqrt
            size = sqrt(len(puzzel))
            if not type(size) == type(1):
                return ('given puzzel not square, puzzel')
        except:
            return ("input not a list of strings, try other input puzzel")
    #size = 10 #voorlopig slechts 10 x 10
    return "L16"
    """
    puzzel = [el for el in filter(lambda x:x in '01.', puzzel)]
    if tussenresultaat:
        print("first row =", puzzel[0:9], "\ntotal = " ,
              len(puzzel))
    #PKHG>DBG OK return was for debugging
    vs = [(r,c) for r in range(size) for c in range(size)]
    v  = [[z3.BitVec('v%d%d' % (r,c),8)  for c in range(size)] for r in range(size)]
    s = z3.Solver()
    
    #convert "." to not yet known int
    for ((r,c), val) in zip(vs, puzzel):
        if val == '.':  
            s.add(0 <= v[r][c], v[r][c] <= 1)
        else:
            s.add(v[r][c] == (int(val)))
            
    #helpers for rows and columns  
    #see description from the given link
    
    def row(n):
        return [(n, i) for i in range(size)]
    def col(n):
        return [(i, n) for i in range(size)]

    for i in range(size):
        s.add(z3.Sum([v[r][c] for r,c in row(i)]) == (size/2))
        s.add(z3.Sum([v[r][c] for r,c in col(i)]) == (size/2))
        
    def row3(n,m):   return [(n, i) for i in range(m,m+3)]
    def col3(n,m):   return [(i, n) for i in range(m,m+3)]
    
    for i in range(size):
        for j in range(size - 2):
            s.add(z3.Sum([v[r][c] for r,c in row3(i,j)]) <= 2)
            s.add(z3.Sum([v[r][c] for r,c in row3(i,j)]) >= 1)
            s.add(z3.Sum([v[r][c] for r,c in col3(i,j)]) <= 2)
            s.add(z3.Sum([v[r][c] for r,c in col3(i,j)]) >= 1)
    #print("L37") dbg info
    s.add(z3.Distinct([z3.Concat([v[r][c] for r,c in row(i)]) for i in range(size)]))
    s.add(z3.Distinct([z3.Concat([v[r][c] for r,c in col(i)]) for i in range(size)]))
    #print("L40")
    print(s.check())
    if s.check() == "unsat":
        #print("assertions = ", s.assertions())
        #return s
        return("geen oplossing (eendudig?) mogelijk")   
    m  = s.model()
    sol = [[m[v[r][c]] for c in range(size)] for r in range(size)]

    gg =  []
    for i in range(10):
        rij = []
        for el in vs[0:10]:
            rij.append(sol[i][el[1]])
        gg.append(rij)
    print("the solution became = \n")
    for i in range(size):
        print(i, gg[i])
    
    #return s

In [7]:
BPsolverZ3(TestBP)

taken from 
          https://pclubiiti.github.io/blog/Inroduction-toSAT-solving/
first row = ['.', '.', '1', '0', '.', '.', '.', '.', '.'] 
total =  100
sat
the solution became = 

0 [1, 0, 1, 0, 1, 1, 0, 0, 1, 0]
1 [0, 0, 1, 0, 1, 0, 1, 0, 1, 1]
2 [0, 1, 0, 1, 0, 0, 1, 1, 0, 1]
3 [1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
4 [1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
5 [0, 1, 0, 0, 1, 0, 1, 1, 0, 1]
6 [0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
7 [1, 1, 0, 1, 0, 1, 0, 1, 0, 0]
8 [1, 1, 0, 0, 1, 0, 1, 0, 1, 0]
9 [0, 0, 1, 1, 0, 1, 0, 1, 0, 1]


In [8]:
BPD.tubantia15aug

['1..1..11..',
 '..........',
 '0.11.11.0.',
 '0.1...0..1',
 '...0......',
 '1.1.......',
 '.0........',
 '....11..0.',
 '11..11...0',
 '.......1..']

# www https://ericpony.github.io/z3py-tutorial/guide-examples.htm

Bevat meer, maar ook de BP-solver werkwijze

# even een check of z3 er is

x = Int('x')
y = Int('y')
print(simplify(x + y + 2*x + 3))
print(simplify(x < y + x + 2))
print(simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)))


x<sup>2</sup> + y<sup>2</sup> &ge; 1

In [9]:
'OK'

'OK'